# Description of Goals
## Description
We are investigatig a Kickstarter datatset that includes properties of Kickstarter campaigns starting from December 2016 until Janurary 2018. The dataset includes 378,000 projects.  The dataset includes the following key properties: 
    date the campaign was launched
    the type of project
    which country it was launch from
    goal amount
    number of backers
    successful funding states
    date of campaign closing
    total amount pledged
    
We are looking for patterns of successfully funded projects among different projects. We want to see what characteristics of a project can help create high likelihood of being funded.
## Importance of Problem
Kickstarter is becoming an increasingly populat platform for startups. For new companies that cannot find or do not want Venture Capital funding or debt, Kickstarter provides an alternative for raising funds. Kickstarter helped kickoff a new era of "Crowdfunding", and continues to be the most popular crowdfunding platform for entrenpurs. We wanted to examine this dataset, because we want to help entrepenurs fund thier new company or product. A Kickstarter campaign takes a lot of time and money, and increasing numbers of people are turning to Kickstarter for funding. Therefore, if a company could know predict if their project would be successful or not before creating a campaign, they could find a better way of fundraising. 

In [9]:
from pandas import Series, DataFrame
import pandas as pd
from patsy import dmatrices
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn import neighbors, metrics, model_selection, naive_bayes
from datetime import datetime
from numpy import exp
import warnings
warnings.filterwarnings('ignore')
# %pylab inline

def is_currency(s):
    '''Takes in an element of the series and outputs if it is a currency'''
    return (s.isupper() and len(s) == 3)

# Create the currency conversions for both pledged and goal
# Conversion valid April 10, 2018 @ 12:00 PM Central Time
currency_conversions = {
    'USD': 1.000000,
    'GBP': 1.41540,
    'CAD': 0.793289,
    'EUR': 1.23490,
    'AUD': 0.776080,
    'SEK': 0.120317,
    'NZD': 0.736415,
    'DKK': 0.165835,
    'NOK': 0.128176,
    'CHF': 1.04558,
    'MXN': 0.0547386,
    'SGD': 0.763829,
    'HKD': 0.127393,
    'JPY': 0.00931953
}

def currency_conversion(df):
    '''Takes in an df and outputs the conversion'''
    for k,v in currency_conversions.iteritems():
        
        # Create a make for the currency
        mask_currency = (df['currency'] == k)
        df.loc[mask_currency, 'usd_goal_current'] = df['goal'] * v
        df.loc[mask_currency, 'usd_pledged_current'] = df['pledged']  * v
    return df

def clean_states(e):
    '''Takes in an elements and returns if it is in a clean state'''
    return (e in ['failed', 'successful', 'suspended', 'canceled'])


def get_month(e):
    '''Takes in a datetime element and returns the month'''
    return e.month

def name_month(e):
    '''Takes in an element from a series, and outputs the 
    string representation of the month'''

    month = int(e)
    return months[month]

months = {
    1: 'January',
    2: 'February',
    3: 'March',
    4: 'April',
    5: 'May',
    6: 'June',
    7: 'July',
    8: 'August',
    9: 'September',
    10: 'October',
    11: 'November',
    12: 'December'
}

def get_days(e):
    '''Takes in a DateTime element from a series and 
    returns the number of days'''
    return e.days

# Load the csv
df = pd.read_csv('ks-projects-201801.csv')

# Drop countries with name N,0"
mask_country_drop = (df['country'] != 'N,0"')
df = df[mask_country_drop]

# Convert the currency into today's price
df = currency_conversion(df)

# Convert the deadline and launched columns into dtype: datetime
df['deadline'] = pd.to_datetime(df['deadline'], format='%Y-%m-%d')
df['launched'] = pd.to_datetime(df['launched'], format='%Y-%m-%d %H:%M:%S')

# Add a 'month' and 'year' coloumn to the df
df['month_deadline'] = df['deadline'].map(get_month)
df['month_launched'] = df['launched'].map(get_month)

# Add an average pledge per backer
df['average_pledged'] = df['pledged'] / df['backers']

# Create a cateogy for the month launched, goals, and quarter launched
df['month_named'] = df['month_launched'].map(name_month).astype('category')

# Bin goal amount
df['goal_binned'] = pd.qcut(df['usd_goal_real'], 10)

# Bin the months into quarters
df['quarter_binned'] = "Q1"
mask_q1 = (df['month_launched'] <= 3)
mask_q2 = ((df['month_launched'] > 3) & (df['month_launched'] <= 6))
mask_q3 = ((df['month_launched'] > 6) & (df['month_launched'] <= 9))
mask_q4 = (df['month_launched'] > 9)

df.loc[mask_q1, 'quarter_binned'] = "Q1"
df.loc[mask_q2, 'quarter_binned'] = "Q2"
df.loc[mask_q3, 'quarter_binned'] = "Q3"
df.loc[mask_q4, 'quarter_binned'] = "Q4"
df['quarter_binned'] = df['quarter_binned'].astype('category')

# Get the length of the project
df['project_length'] = df['deadline'] - df['launched']
df['project_length_days'] = df['project_length'].map(get_days)
df['project_length_days_binned'] = pd.qcut(df['project_length_days'], 3)
# print df['project_length_days'].value_counts()

# Get a baseline for the ML
# The first part is going to predict based off of cateogrical variables: 
# category, month-launched, month-deadline

# There are 6 states: failed, successful, canceled, undefied, lived, suspended
# For classification purposes 1.0 = successful, 0.0 = failed, suspended, canceled
# Create a mask of just the clean states
mask_clean_states = (df['state'].map(clean_states))
df_clean_states = df[mask_clean_states]

# Get the value counts for the clean states
vc_clean_states = df_clean_states['state'].value_counts()

# Find the baseline by dividing the total of project by the number of successful projects
# baseline = 0.359806607575
num_total_projects = vc_clean_states.sum()
num_successful_projects = vc_clean_states['successful']
baseline = float(num_successful_projects) / float(num_total_projects)

# Create a dummy binary variable for success or fail/cancel/suspended
df_clean_states['target'] = 0.0
mask_target = df_clean_states['state'] == 'successful'
df_clean_states.loc[mask_target, 'target'] = 1.0

0         (31.0, 16738.0]
1         (31.0, 16738.0]
2         (31.0, 16738.0]
3          (-0.001, 29.0]
4         (31.0, 16738.0]
5         (31.0, 16738.0]
6          (-0.001, 29.0]
7         (31.0, 16738.0]
8         (31.0, 16738.0]
9          (-0.001, 29.0]
10         (-0.001, 29.0]
11         (-0.001, 29.0]
12         (-0.001, 29.0]
13        (31.0, 16738.0]
14         (-0.001, 29.0]
15         (-0.001, 29.0]
16         (-0.001, 29.0]
17         (-0.001, 29.0]
18         (-0.001, 29.0]
19         (-0.001, 29.0]
20        (31.0, 16738.0]
21         (-0.001, 29.0]
22         (-0.001, 29.0]
23         (-0.001, 29.0]
24         (-0.001, 29.0]
25         (-0.001, 29.0]
26         (-0.001, 29.0]
27         (-0.001, 29.0]
28         (-0.001, 29.0]
29         (-0.001, 29.0]
               ...       
378631     (-0.001, 29.0]
378632     (-0.001, 29.0]
378633     (-0.001, 29.0]
378634    (31.0, 16738.0]
378635     (-0.001, 29.0]
378636     (-0.001, 29.0]
378637    (31.0, 16738.0]
378638      

# Solution and insights
## KNN
Our problem, is a classification problem. We want to use certain features of a Kickstarter Campaign to determine if the campagin was successful or not. We first had to clean the data and convert the columns into proper datetime objects. We then got our baseline for the dataset, which is 63.55% for only picking that a project will not get funding.

In the dataset there is a feature titled "state", which says the status of the project. There are six possible states, but we are only concerned with four of them: successful, failed, canceled, and suspended. Our first approach was to use a k-Nearest-Neighbors (kNN) classifer to make predictions. In our approach we created a new target column and assigned a binary value. 1.0 was assigned for successful projects, and 0.0 was assinged for the other three states. 

In our first attempt we chose two features: number of backers and target goal converted to USD according to the date of the project launch. We used cross-validation to determine the optimal number of nearest-neighbors. Surpringly, the number of nearest-neighbors for numbers under 100 has about the same accuracy. It ranged from 91% to 93% accurate. Increasing the number of nearest neighbors more than 100 say decreased accuracy. 

(graph) Although we had really high accuracy with this kNN classifer, analyzing the model shows it is not the best option. We originally thought knowing the how many backers you need compared to your goal would help you determine how to run your project. Looking at the graph, you can see that there are two clusters of projects. Those with high backers and lower goals that succeeded, and those with little backers and higher goals that failed. This classifer is fairly intuitive and relies on data from after the project has finished. Since we want to help future Kickstarter projects. We felt that we could do better using data before the project is launched.

In [ ]:
# Start with simple KNN with two features: Number of Backers and Goal Amount
def accuracy(X, y, nn):

    model = neighbors.KNeighborsClassifier(n_neighbors=nn, weights='uniform')
    accuracies = []

    kfold = model_selection.StratifiedKFold(n_splits=3, shuffle=True).split(X, y)
    for train, holdout in kfold:
        # Select the training and testing data using the indicies from kfold
        X_train = X.iloc[train]
        X_holdout = X.iloc[holdout]
        y_train = y[train]
        y_holdout = y[holdout]

        # Fit the model
        model.fit(X_train, y_train)

        # Compute the accuracy
        current_accuracy = metrics.accuracy_score(y_holdout, model.predict(X_holdout))
        accuracies.append(current_accuracy)
        
    return accuracy


# Create a design matrice
Y, X = dmatrices('target ~ 0 + backers + usd_goal_real', df_clean_states, return_type='dataframe')
y = Y['target'].values
# sc = scatter(df_clean_states['backers'], df_clean_states['usd_goal_real'], c=y, cmap='bwr')


# Create sample nn
nn = [15]
scores = [accuracy(X, y, num_nbrs) for num_nbrs in nn]
print scores

# Best Nearest Neighbor Numbers: Up to 100 nn they were all about the same accuracy between 91% and 93% 
# So you are best just using around 15 backers becuase it is faster
# Starts to see small dips over 100 nn
# Looks like the more backers and lower goal means that you have the best shot
# But that makes sense, and is only after the fact
# Backers is continuous not categorical

## Naive Bayes
Since most of the data seems independent and categorical, the Naive-Bayes classifer seemed like a better choice. There is a correlation between category/main category and goal (figure), with different categories having higher average goals. However, there is a wide spread of goals in each category and therefore it is hard to true determine how dependent the variables are. 

We created five categorical parameters: goal amounts in 10 bins, length of fundraising in three bins, category, country, and the quarter the project was started. The three bins for the length are less than 29 days, between 29 and 31 days, and over 31 days. With most of the data between 29 and 31 days.

With the Naive Bayes model we were able to get 68.92% accuracy. This is only slighty higher than the baseline of 63.55%. The class priors are slightly different than the baseline we predicted but not too much. Looking at the feature importance gives us further insight to what makes a Kickstarter project successful. 9 out of the 10 top important features are categories, and 4 of the featuers belong to the main category of technology. The only other non-category feature is the goal binned. This shows that the projects with reasonable funding goals get are important in this model for project successfulness. 

In [11]:
# What about the category, main category, country, month launched, and goal amount
# Can we predict if the project will be successfully funded
# We first need to discretize the goal amount
# Create 20 bins of the goal amount
# usd_goal_real is discrete not catoegorical
def accuracy_nb(X, y, splits, model):
    
    accuracies = []

    kfold = model_selection.StratifiedKFold(n_splits=splits, shuffle=True).split(X, y)
    for train, holdout in kfold:
        # Select the training and testing data using the indicies from kfold
        X_train = X.iloc[train]
        X_holdout = X.iloc[holdout]
        y_train = y[train]
        y_holdout = y[holdout]

        # Fit the model
        model.fit(X_train, y_train)

        # Compute the accuracy
        current_accuracy = metrics.accuracy_score(y_holdout, model.predict(X_holdout))
        accuracies.append(current_accuracy)
        
    return accuracies

# Create a list of categorical columns that will be used
categorical_columns = ['goal_binned', 'category',
'country', 'quarter_binned', 'project_length_days_binned']

# Create a df that includes the dummy variables (0/1) for all categorical data
df_dummies = pd.get_dummies(df_clean_states[categorical_columns],
prefix=categorical_columns,columns=categorical_columns)

# Concatenate all dummy columns into the dataframe
df_clean_dummies = pd.concat([df_clean_states, df_dummies], axis=1)

# Create the formula
formula = 'target ~ 0 + {}'.format(' + '.join(['Q("{}")'.format(x) for x in df_dummies.columns.values[:205]]))

# Create the testings and training matrices
Y,X = dmatrices(formula, df_clean_dummies, return_type='dataframe')
y = Y['target'].values
    
# Build and run the model
model = naive_bayes.MultinomialNB()
accuracies = accuracy_nb(X, y, 3, model)
print accuracies

# Get the class priors
print 'Negative Class Prior'
print exp(model.class_log_prior_[0])
print 'Positive Class Prior'
print exp(model.class_log_prior_[1])

# Check class priors
print
print 'Check class priors'
print df_clean_dummies['target'].value_counts() / len(df)

# Check the likelihood and feature importance
# Seems like technology categories are the most important...
feature_importance = abs(model.feature_log_prob_[1] - model.feature_log_prob_[0])
series_feature_importance = Series(feature_importance, index=X.columns.values)
print series_feature_importance.sort_values(ascending=False)[:10]

# Need to build up cross validation

# Get the categories with the most funding
temp = df.groupby('main_category')['usd_goal_real'].mean().sort_values(ascending=False)


# Maybe use the test data for the live ones?


# Possible Features:
# Month launched and deadline
# Goal Amount
# Number of Backers
# Country of Origin
# Category and Main Category

[0.68921642934318106, 0.68753124445662861, 0.68847462546967475]
Negative Class Prior
0.640249310606
Positive Class Prior
0.359750689394

Check class priors
0.0    0.635470
1.0    0.357065
Name: target, dtype: float64
Q("category_Apps")                            2.167262
Q("category_Chiptune")                        1.962546
Q("category_Web")                             1.804772
Q("category_Mobile Games")                    1.726333
Q("category_Letterpress")                     1.632305
Q("category_Video")                           1.406776
Q("category_Residencies")                     1.402931
Q("goal_binned_(46049.259, 166361390.71]")    1.392897
Q("category_Software")                        1.361690
Q("category_Candles")                         1.332232
dtype: float64
